# Neural Networks

## Perceptron

In [2]:
import numpy as np

np.random.seed(812)

# First col of inputs matches output
inputs = np.array([
    [0, 0, 1],
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1]
])

correct_outputs = [[0], [1], [1], [0]]

In [4]:
# Activation Function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [5]:
# Keep weights bounded [-1, 1]
weights = 2 * np.random.random((3, 1)) - 1
weights

array([[ 0.0099616 ],
       [ 0.21185521],
       [-0.08502562]])

In [6]:
# Steps
# 1. Randomly Initialized Weights
# 2. Got input data & correct_outputs

# Update 10,000 times - iterations or "epochs"
for iteration in range(10000):
    
    # Weighted sum of inputs
    weighted_sum = np.dot(inputs, weights)
    
    # Activate with Sigmoid
    activated_output = sigmoid(weighted_sum)
    
    # Cac error
    error = correct_outputs - activated_output
    adjustments = error * sigmoid_derivative(activated_output)
    
    # Update weights
    weights += np.dot(inputs.T, adjustments)
    
print("Weights after training")
print(weights)

print ('')

print("Output after training")
print(activated_output)

Weights after training
[[15.03804491]
 [-0.40666422]
 [-7.23278107]]

Output after training
[[7.22059439e-04]
 [9.99388204e-01]
 [9.99592541e-01]
 [4.80912067e-04]]


## Perceptron Class

In [7]:
class Perceptron(object):
    def __init__(self, rate = 0.01, niter = 10):
    self.rate = rate
    self.niter = niter

    def fit(self, X, y):
    """Fit training data
    X : Training vectors, X.shape : [#samples, #features]
    y : Target values, y.shape : [#samples]
    """

    # weights
    self.weight = np.zeros(1 + X.shape[1])

    # Number of misclassifications
    self.errors = []  # Number of misclassifications

    for i in range(self.niter):
      err = 0
      for xi, target in zip(X, y):
        delta_w = self.rate * (target - self.predict(xi))
        self.weight[1:] += delta_w * xi
        self.weight[0] += delta_w
        err += int(delta_w != 0.0)
      self.errors.append(err)
    return self

  def net_input(self, X):
    """Calculate net input"""
    return np.dot(X, self.weight[1:]) + self.weight[0]

  def predict(self, X):
    """Return class label after unit step"""
    return np.where(self.net_input(X) >= 0.0, 1, -1)

In [8]:
# I want activations that correspond to negative weights to be lower
# and activations that correspond to positive weights to be higher

class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 2
        self.hiddenNodes = 3
        self.output = 1

        # Initial Weights
        # 2x3 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.output)
    
    # Define activation function
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    # Slope of activation function
    def sigmoidPrime(self, s):
        sx = self.sigmoid(s)
        return sx * (1 - sx)
    
    # Feed forward / prediction method
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka 'predict'
        """
        
        # Weighted sum of inputs
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of Weighted Sums
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Next layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final node - activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
    
    def backward(self, X, y, y_pred):
        """
        Back propagate through the network
        """
        
        # Error in output
        self.output_error = y - y_pred
        
        
        # hidden --> Output
        # Apply Derivative of Sigmoid to error
        self.output_delta = self.output_error * self.sigmoidPrime(y_pred)
        
        # z2 Error
        self.z2_error = self.output_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Update weights
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjust second set of weights
        self.weights2 += self.activated_hidden.T.dot(self.output_delta)
        
    def train(self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

## Tensorflow & Keras

In [9]:
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

np.random.seed(812)

In [10]:
model = Sequential()

# Add a dense layer
model.add(Dense(1, input_dim=8, activation="sigmoid"))

# Define loss function and compile
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.fit(X, y, epochs=10)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


NameError: name 'X' is not defined

In [ ]:
# evaluate the model
scores = model.evaluate(X, y)
print(f"{model.metrics_names[1]}: {scores[1]*100}")

## K-Fold Validation

In [13]:
import pandas as pd

In [14]:
# Keras imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# sklearn imports
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Random Seed
seed = 42
np.random.seed(seed)

# Load Dataset
boston_dataset = load_boston()
df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target

# Split into X and y and turn into numpy arays
y = df.MEDV.values
X = df.drop("MEDV", axis='columns').values

# Important Hyperparameters
inputs = X.shape[1]
epochs = 50
batch_size = 10

# define base model
def baseline_model():
  # create model
  model = Sequential()
  model.add(Dense(64, input_dim=inputs, activation='relu')) 
  model.add(Dense(64, activation='relu')) 
  model.add(Dense(1))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=epochs, batch_size=batch_size,
    verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(f"K-fold Cross-Val Results - Mean: {-results.mean():.2f} StDev: {results.std():.2f} MSE")

K-fold Cross-Val Results - Mean: 22.33 StDev: 13.10 MSE


## Grid CV Search

In [15]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load dataset
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/20
512/512 [==============================] - 0s 337us/sample - loss: 7.2902 - acc: 0.5957
Epoch 2/20
512/512 [==============================] - 0s 238us/sample - loss: 4.4856 - acc: 0.5410
Epoch 3/20
512/512 [==============================] - 0s 261us/sample - loss: 2.5586 - acc: 0.5449
Epoch 4/20
512/512 [==============================] - 0s 254us/sample - loss: 1.8141 - acc: 0.5645s - loss: 1.8774 - acc: 0.554
Epoch 5/20
512/512 [==============================] - 0s 204us/sample - loss: 1.6869 - acc: 0.5820
Epoch 6/20
512/512 [==============================] - 0s 155us/sample - loss: 1.6259 - acc: 0.5840
Epoch 7/20
512/512 [==============================] - 0s 210us/sample - loss: 1.5118 - acc: 0.5918
Epoch 8/20
512/512 [==============================] - 0s 193us/sample - loss: 1.4505 - acc: 0.5898
Epoch 9/20
512/512 [==============================] - 0s 184us/sample - loss: 1.3531 - acc: 0.5879
Epoch 10/20
512/512 [==============================] - 0s 184us/sample - loss: 1

512/512 [==============================] - 0s 156us/sample - loss: 2.0809 - acc: 0.6641
Epoch 18/20
512/512 [==============================] - 0s 147us/sample - loss: 1.9960 - acc: 0.6641
Epoch 19/20
512/512 [==============================] - 0s 90us/sample - loss: 1.9072 - acc: 0.6797
Epoch 20/20
512/512 [==============================] - 0s 58us/sample - loss: 1.7979 - acc: 0.6914
Epoch 1/20
512/512 [==============================] - 0s 494us/sample - loss: 26.7298 - acc: 0.6426
Epoch 2/20
512/512 [==============================] - 0s 113us/sample - loss: 16.3755 - acc: 0.6230
Epoch 3/20
512/512 [==============================] - 0s 81us/sample - loss: 9.5057 - acc: 0.5000
Epoch 4/20
512/512 [==============================] - 0s 61us/sample - loss: 8.0229 - acc: 0.4785
Epoch 5/20
512/512 [==============================] - 0s 83us/sample - loss: 7.2650 - acc: 0.4922
Epoch 6/20
512/512 [==============================] - 0s 97us/sample - loss: 6.6295 - acc: 0.4844
Epoch 7/20
512/512 [==

512/512 [==============================] - 0s 116us/sample - loss: 2.2801 - acc: 0.5352
Epoch 13/20
512/512 [==============================] - 0s 82us/sample - loss: 2.1372 - acc: 0.5488
Epoch 14/20
512/512 [==============================] - 0s 40us/sample - loss: 1.9980 - acc: 0.5566
Epoch 15/20
512/512 [==============================] - 0s 137us/sample - loss: 1.9082 - acc: 0.5625
Epoch 16/20
512/512 [==============================] - 0s 237us/sample - loss: 1.8324 - acc: 0.5488s - loss: 1.8142 - acc: 0.556
Epoch 17/20
512/512 [==============================] - 0s 194us/sample - loss: 1.7501 - acc: 0.5625s - loss: 1.7198 - acc: 0.575
Epoch 18/20
512/512 [==============================] - 0s 149us/sample - loss: 1.6902 - acc: 0.5684
Epoch 19/20
512/512 [==============================] - 0s 212us/sample - loss: 1.6299 - acc: 0.5762
Epoch 20/20
512/512 [==============================] - 0s 27us/sample - loss: 1.5273 - acc: 0.5742
Epoch 1/20
512/512 [==============================] - 0s 

512/512 [==============================] - 0s 142us/sample - loss: 3.3375 - acc: 0.6289
Epoch 7/20
512/512 [==============================] - 0s 48us/sample - loss: 2.1849 - acc: 0.6270
Epoch 8/20
512/512 [==============================] - 0s 80us/sample - loss: 1.7779 - acc: 0.6465
Epoch 9/20
512/512 [==============================] - 0s 126us/sample - loss: 1.6845 - acc: 0.6270
Epoch 10/20
512/512 [==============================] - 0s 143us/sample - loss: 1.5600 - acc: 0.6367
Epoch 11/20
512/512 [==============================] - 0s 70us/sample - loss: 1.4634 - acc: 0.6328
Epoch 12/20
512/512 [==============================] - 0s 134us/sample - loss: 1.4229 - acc: 0.6582
Epoch 13/20
512/512 [==============================] - 0s 79us/sample - loss: 1.3517 - acc: 0.6348
Epoch 14/20
512/512 [==============================] - 0s 176us/sample - loss: 1.3018 - acc: 0.6426
Epoch 15/20
512/512 [==============================] - 0s 124us/sample - loss: 1.2489 - acc: 0.6484
Epoch 16/20
512/512

512/512 [==============================] - 0s 360us/sample - loss: 10.1789 - acc: 0.5137
Epoch 2/20
512/512 [==============================] - 0s 24us/sample - loss: 9.2615 - acc: 0.5234
Epoch 3/20
512/512 [==============================] - 0s 51us/sample - loss: 8.4033 - acc: 0.5215
Epoch 4/20
512/512 [==============================] - 0s 42us/sample - loss: 7.6051 - acc: 0.5195
Epoch 5/20
512/512 [==============================] - 0s 40us/sample - loss: 7.0339 - acc: 0.5098
Epoch 6/20
512/512 [==============================] - 0s 34us/sample - loss: 6.3665 - acc: 0.5098
Epoch 7/20
512/512 [==============================] - 0s 40us/sample - loss: 5.8305 - acc: 0.5352
Epoch 8/20
512/512 [==============================] - 0s 19us/sample - loss: 5.4404 - acc: 0.5371
Epoch 9/20
512/512 [==============================] - 0s 62us/sample - loss: 5.0507 - acc: 0.5410
Epoch 10/20
512/512 [==============================] - 0s 55us/sample - loss: 4.7559 - acc: 0.5430
Epoch 11/20
512/512 [=======

768/768 [==============================] - 0s 184us/sample - loss: 0.9736 - acc: 0.6302
Epoch 19/20
768/768 [==============================] - 0s 165us/sample - loss: 0.9192 - acc: 0.6276
Epoch 20/20
768/768 [==============================] - 0s 161us/sample - loss: 0.9641 - acc: 0.6276
Best: 0.6184895833333334 using {'batch_size': 10, 'epochs': 20}
Means: 0.6184895833333334, Stdev: 0.014381980491129247 with: {'batch_size': 10, 'epochs': 20}
Means: 0.6028645833333334, Stdev: 0.0433028363237393 with: {'batch_size': 20, 'epochs': 20}
Means: 0.5520833333333334, Stdev: 0.03962271889245557 with: {'batch_size': 40, 'epochs': 20}
Means: 0.5677083333333334, Stdev: 0.038051143740110566 with: {'batch_size': 60, 'epochs': 20}
Means: 0.5078125, Stdev: 0.05063078670631141 with: {'batch_size': 80, 'epochs': 20}
Means: 0.5768229166666666, Stdev: 0.017566064535458385 with: {'batch_size': 100, 'epochs': 20}
